<a href="https://colab.research.google.com/github/Geunju-hub/2022-Samsung-AI-Challenge-3D-Metrology-/blob/main/%5BBaseline%5D_Depth_Map_%EC%83%9D%EC%84%B1_AutoEncoder_ipynb_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D Metrology


* Train Dataset (학습용 데이터셋, 학습 가능) - 총 60664개
SEM [폴더] : 실제 SEM 영상을 Hole 단위로 분할한 영상 (8bit Gray 영상)
average_depth.csv : 전체 SEM 영상과 대응되는 평균 Depth


* Simulation Dataset (학습용 데이터셋, 학습 가능) - 총 259956개
  * SEM [폴더] : Simulator을 통해 생성한 Hole 단위 SEM 영상 (실제 SEM 영상과 유사하나, 대응 관계는 없음)
  * Depth [폴더] : Simulator을 통해 얻은 SEM 영상과 Pixel별로 대응되는 Depth Map
Depth 이미지 1개당 2개의 Simulator Hole 단위 SEM 영상이 Pair하게 매칭됩니다. (Name_itr0, Name_itr1)


* Test Dataset (평가를 위한 테스트 데이터셋, 학습 불가능) - 총 25998개
SEM [폴더] : 실제 SEM 영상을 Hole 단위로 분할한 영상 (8bit Gray 영상)


* sample_submission.zip (제출 양식) - 총 25998개
실제 Hole 단위 SEM 영상으로부터 추론한 Depth Map (.png, int)

## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -qq "/content/drive/MyDrive/공유 드라이브/삼성데이콘/open.zip"    # 로컬

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/공유 드라이브/삼성데이콘/average_depth.csv")  # 경로 확인!!
df.head()

,0,1
0,depth_140_site_00233,138.966170
1,depth_140_site_00272,139.588011
2,depth_140_site_00183,137.022387
3,depth_140_site_00204,138.401329
4,depth_140_site_00187,139.402107


## Hyperparameter Setting

In [ ]:
CFG = {
    'WIDTH':48,
    'HEIGHT':72,
    'EPOCHS':10,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':128,
    'SEED':41
}

## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [ ]:
simulation_sem_paths = sorted(glob.glob('./simulation_data/SEM/*/*/*.png'))
simulation_depth_paths = sorted(glob.glob('./simulation_data/Depth/*/*/*.png')+ glob.glob('./simulation_data/Depth/*/*/*.png'))

In [ ]:
print(simulation_sem_paths[:10])

['./simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0005-01MS_3_itr0.png', './simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0005-01MS_3_itr1.png', './simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0006-01MS_3_itr0.png', './simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0006-01MS_3_itr1.png', './simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0009-01MS_3_itr0.png', './simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0009-01MS_3_itr1.png', './simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0029-01MS_1_itr0.png', './simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0029-01MS_1_itr1.png', './simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0030-01MS_2_itr0.png', './simulation_data/SEM/Case_1/80/20201001_202940_NE142400C_RAE01_1_S01_M0030-01MS_2_itr1.png']


In [ ]:
data_len = len(simulation_sem_paths) # == len(simulation_depth_paths)

In [ ]:
data_len

173304

In [ ]:
train_sem_paths = simulation_sem_paths[:int(data_len*0.8)]
train_depth_paths = simulation_depth_paths[:int(data_len*0.8)]

val_sem_paths = simulation_sem_paths[int(data_len*0.8):]
val_depth_paths = simulation_depth_paths[int(data_len*0.8):]

## CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, sem_path_list, depth_path_list):
        self.sem_path_list = sem_path_list
        self.depth_path_list = depth_path_list
        
    def __getitem__(self, index):
        sem_path = self.sem_path_list[index]
        sem_img = cv2.imread(sem_path, cv2.IMREAD_GRAYSCALE)                    # rgb 3이여서 gray로 변환
        #clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize = (8,8))
        #sem_img = clahe.apply(sem_img)                                          # clahe 사용
        sem_img = np.expand_dims(sem_img, axis=-1).transpose(2,0,1)             # 
        sem_img = sem_img / 255.                                                # 표준화
        
        if self.depth_path_list is not None:
            depth_path = self.depth_path_list[index]
            depth_img = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)
            #depth_img = clahe.apply(depth_img)                                  # clahe 사용
            depth_img = np.expand_dims(depth_img, axis=-1).transpose(2,0,1)
            depth_img = depth_img / 255.
            return torch.Tensor(sem_img), torch.Tensor(depth_img) # B,C,H,W     # batch, channel, height, width
        else:
            img_name = sem_path.split('/')[-1]
            return torch.Tensor(sem_img), img_name # B,C,H,W
        
    def __len__(self):
        return len(self.sem_path_list)

In [ ]:
train_dataset = CustomDataset(train_sem_paths, train_depth_paths)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=6)

val_dataset = CustomDataset(val_sem_paths, val_depth_paths)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=6)

## Model Define

In [ ]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(CFG['HEIGHT']*CFG['WIDTH'], 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(), 
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, CFG['HEIGHT']*CFG['WIDTH'])
        )
        
    def forward(self, x):
        x = x.view(-1, CFG['HEIGHT']*CFG['WIDTH'])
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.view(-1, 1, CFG['HEIGHT'], CFG['WIDTH'])
        return x

In [ ]:
class CNN_Model(nn.Module):
  def __init__(self):
    super(CNN_Model, self).__init__()
       # Encoder
    self.cnn_layer1 = nn.Sequential(
                        nn.Conv2d(1, 128, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(128),
                        nn.ReLU(),
                        nn.MaxPool2d(2,2))

    self.cnn_layer2 = nn.Sequential(
                                nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                                nn.BatchNorm2d(256),
                                nn.ReLU(),
                                nn.MaxPool2d(2,2))
    self.cnn_layer3 = nn.Sequential(
        nn.Conv2d(256,512,kernel_size = 3,stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
        # Decoder
    self.tran_cnn_layer1 = nn.Sequential(
                        nn.ConvTranspose2d(512, 256, kernel_size = 2, stride = 2, padding=0),
                        nn.BatchNorm2d(256),
                        nn.ReLU())

    self.tran_cnn_layer2 = nn.Sequential(
                        nn.ConvTranspose2d(256, 128, kernel_size = 2, stride = 2, padding=0),
                        nn.BatchNorm2d(128),
                        nn.ReLU(),
                        )
    self.tran_cnn_layer3 = nn.Sequential(
        nn.ConvTranspose2d(128,1,kernel_size = 2, stride = 2, padding = 0),
        nn.Sigmoid()
    )        
            
  def forward(self, x):
    output = self.cnn_layer1(x)
    output = self.cnn_layer2(output)
    output = self.cnn_layer3(output)
    output = self.tran_cnn_layer1(output)
    output = self.tran_cnn_layer2(output)
    output = self.tran_cnn_layer3(output)
    return output

## Train

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.L1Loss().to(device)
    best_score = 999999
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for sem, depth in tqdm(iter(train_loader)):
            sem = sem.float().to(device)
            depth = depth.float().to(device)
            
            optimizer.zero_grad()
            
            model_pred = model(sem)
            
            loss = criterion(model_pred, depth)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        val_loss, val_rmse = validation(model, criterion, val_loader, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] Val RMSE : [{val_rmse:.5f}]')
        
        if best_score > val_rmse:
            best_score = val_rmse
            best_model = model
        
        if scheduler is not None:
            scheduler.step()
            
    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    rmse = nn.MSELoss().to(device)
    
    val_loss = []
    val_rmse = []
    with torch.no_grad():
        for sem, depth in tqdm(iter(val_loader)):
            sem = sem.float().to(device)
            depth = depth.float().to(device)
            
            model_pred = model(sem)
            loss = criterion(model_pred, depth)
            
            pred = (model_pred*255.).type(torch.int8).float()
            true = (depth*255.).type(torch.int8).float()
            
            b_rmse = torch.sqrt(criterion(pred, true))
            
            val_loss.append(loss.item())
            val_rmse.append(b_rmse.item())

    return np.mean(val_loss), np.mean(val_rmse)

## Run!!

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda epoch: 0.95 ** epoch)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/1084 [00:00<?, ?it/s]

  0%|          | 0/271 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.04852] Val Loss : [0.05483] Val RMSE : [4.57556]


  0%|          | 0/1084 [00:00<?, ?it/s]

  0%|          | 0/271 [00:00<?, ?it/s]

...

  0%|          | 0/271 [00:00<?, ?it/s]

Epoch : [9] Train Loss : [0.01075] Val Loss : [0.01614] Val RMSE : [2.48752]


  0%|          | 0/1084 [00:00<?, ?it/s]

  0%|          | 0/271 [00:00<?, ?it/s]

Epoch : [10] Train Loss : [0.01029] Val Loss : [0.01279] Val RMSE : [2.28414]


## Inference & Submission

In [ ]:
# 모델 저장
torch.save(infer_model, "/content/drive/MyDrive/Dacon/base_line_plus_scheduler.pt")

In [ ]:
infer_model = torch.load("/content/drive/MyDrive/Dacon/base_line_plus_scheduler.pt")

In [ ]:
test_sem_path_list = sorted(glob.glob('./test/SEM/*.png'))

In [ ]:
test_dataset = CustomDataset(test_sem_path_list, None)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=6)

In [ ]:
import zipfile
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    result_name_list = []
    result_list = []
    with torch.no_grad():
        for sem, name in tqdm(iter(test_loader)):
            sem = sem.float().to(device)
            model_pred = model(sem)
            
            for pred, img_name in zip(model_pred, name):
                pred = pred.cpu().numpy().transpose(1,2,0)*255.
                save_img_path = f'{img_name}'
                #cv2.imwrite(save_img_path, pred)
                result_name_list.append(save_img_path)
                result_list.append(pred)
    
    os.makedirs('./submission', exist_ok=True)
    os.chdir("./submission/")
    sub_imgs = []
    for path, pred_img in zip(result_name_list, result_list):
        cv2.imwrite(path, pred_img)
        sub_imgs.append(path)
    submission = zipfile.ZipFile("../submission.zip", 'w')
    for path in sub_imgs:
        submission.write(path)
    submission.close()

In [ ]:
inference(infer_model, test_loader, device)

  0%|          | 0/204 [00:00<?, ?it/s]